In [1]:
import config

In [2]:
config.print_config()

Configuration:
Data Directory: C:/Users/edvar/PycharmProjects/miniFrance_project/data
Image Size: (1000, 1000)
Number of Classes: 16
Backboneresnet50, is pretrained? True
Devide for torch: cuda


In [2]:
import os
import logging
import numpy as np
import torch
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import  reshape_as_image
from PIL import Image
from torchvision.io import read_image
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import config

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


class MiniFranceDataset(Dataset):
    def __init__(self, transform=None):
        self.image_dir = config.TRAIN_IMAGE_DIR
        self.mask_dir = config.TRAIN_MASK_DIR
        self.image_files = os.listdir(config.TRAIN_IMAGE_DIR)
        self.trasformation = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_filename = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_filename)
        # mask_path = os.path.join(self.mask_dir, f"mask_{os.path.splitext(image_filename)[0]}.tif")

        image = reshape_as_image(rasterio.open(image_path).read())
        # image = ToTensor()(image)
        # mask = Image.open(mask_path).convert('RGB')
        # mask = ToTensor()(mask)

        # if self.trasformation:
        #     image = self.trasformation(image)
        #     # mask = self.transform(mask)

        return image_path
    
data = MiniFranceDataset()
    # print(data.image_files)

print(data[0])
# for i, image in enumerate(data):
#     print(image.size)

C:/Users/edvar/PycharmProjects/miniFrance_project/data\train_images\D006_0990-6305_25.tif


C:\Users\edvar\miniconda3\envs\miniFrance_project\lib\site-packages\rasterio\__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [3]:
image_array = rasterio.open('C:/Users/edvar/PycharmProjects/miniFrance_project/data/train_images\D006_0990-6305_25.tif').read()
image_array = reshape_as_image(image_array)

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(image_array)
plt.show()

In [1]:
import torch

torch.cuda.empty_cache()

In [2]:
torch.cuda.is_available()

True

In [7]:
import hyperparameter_config
for value in hyperparameter_config.__repr__:
    print(f"{value}")

TypeError: 'method-wrapper' object is not iterable

In [8]:
hyperparameter_config.__repr__

<method-wrapper '__repr__' of module object at 0x000001EF1664A160>